<a href="https://colab.research.google.com/github/mint-rin/HUFSdataAnalysis/blob/main/final_project01_NaverMovieReviewCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 영화 리뷰 페이지에 접속하기

In [ ]:
# requests 패키지 불러오기 (pip install requests 설치 필요)
import requests
import time

# 네이버 영화 사이트 - 주토피아 리뷰 페이지
url = "https://movie.naver.com/movie/bi/mi/review.naver?code=130850"

# requests.get
resp = requests.get(url)
print(resp)

<Response [200]>


In [ ]:
# 서버 응답 중에서 소스코드 부분을 출력
resp.text[150:500]

'ng="ko">\r\n<head>\r\n<meta charset="utf-8">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta http-equiv="imagetoolbar" content="no">\r\n<title>주토피아 : 네이버 영화</title>\r\n\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\r\n\t\t\r\n\t\t\r\n\t\r\n\r\n\r\n\r\n<meta property="me2:image" content="https://ssl.pstatic.net/imgmovie/today/naverme/naverme_profile.jpg"/>\r\n<meta property="me2:post_tag" conte'

# BeautifulSoup 으로 HTML 해석하기

In [ ]:
# BeautifulSoup 함수 불러오기 (pip install bs4 설치 필요)
from bs4 import BeautifulSoup

# BeautifulSoup 함수로, HTML 문서 구조를 파싱
soup = BeautifulSoup(resp.text, 'html.parser')

# 파싱한 내용을 담고 있는 객체의 자료형 확인
print(type(soup))

<class 'bs4.BeautifulSoup'>


## 1) 영화 제목

In [ ]:
# <title> 주토피아 : 네이버 영화</title>

# title 태그 이름을 활용하여 영화 제목이 포함되어 있는 요소를 찾습니다.
title_tag = soup.find(name='title')
print(title_tag)

# 텍스트 부분만 추출합니다.
title_text = title_tag.get_text()
print(title_text)

<title>주토피아 : 네이버 영화</title>
주토피아 : 네이버 영화


## 2) 리뷰 개수

In [ ]:
# <span class="cnt">총<em>110</em>건</span>

# span 태그의 class 속성값을 활용하여 리뷰 갯수가 포함되어 있는 요소를 찾습니다.
count_tag = soup.find(name='span', attrs={'class':'cnt'})
print("span 태그: ", count_tag)

# count_tag 요소에서 em 태그 부분을 찾습니다.
count_tag = count_tag.find(name='em')
print("em 태그: ", count_tag)

# 텍스트 부분만 추출합니다.
count_text = count_tag.get_text()
print("텍스트: ", count_text)

span 태그:  <span class="cnt">총<em>1,970</em>건</span>
em 태그:  <em>1,970</em>
텍스트:  1,970


## 3) 리뷰 목록

In [ ]:
# <ul class="rvw_list_area">

# ul 태그의 class 속성값을 활용하여 리뷰 제목과 링크가 포함되어 있는 요소를 찾습니다.
review_list_tag = soup.find(name='ul', attrs={'class':'rvw_list_area'})

# review_list_tag 요소에 포함된 li 태그를 모두 찾습니다.
review_list_tags = review_list_tag.find_all(name='li')
print("li 태그의 수: ", len(review_list_tags))
print("\n")

# 첫 번째 원소를 출력합니다.
print(review_list_tags[0])

li 태그의 수:  10


<li>
<a href="#" onclick="clickcr(this, 'rli.title', '', '', event); showReviewDetail(4209726)"><strong>주토피아 (Zootopia)</strong></a>
<span class="user"><a href="#" onclick="clickcr(this, 'rli.uid', '', '', event); showReviewListByNid('4209726');">trav****</a> <em>2016.02.20</em><em>추천 51</em></span>
<p><a href="#" onclick="clickcr(this, 'rli.content', '', '', event); showReviewDetail(4209726);">★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마...</a></p>
</li>


In [ ]:
# 첫 번째 아이템에서 리뷰 제목(rli.title), 사용자(rli.uid), 내용(rli.content) 데이터를 추출합니다.

review_title = review_list_tags[0].find_all('a')[0].get_text()
print("제목:", review_title, "\n")

review_uid = review_list_tags[0].find_all('a')[1].get_text()
print("사용자:", review_uid, "\n")

review_content = review_list_tags[0].find_all('a')[2].get_text()
print(r"내용:", review_content, "\n")

제목: 주토피아 (Zootopia) 

사용자: trav**** 

내용: ★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마... 



In [ ]:
# 각 리뷰 페이지로 연결할 nid 값(예: 4808692)을 포함하는 Javascript 코드를 추출합니다.

review_nid = review_list_tags[0].find('a').get('onclick')
review_nid

"clickcr(this, 'rli.title', '', '', event); showReviewDetail(4209726)"

In [ ]:
# Javascript 코드에서 7자리 숫자를 추출합니다. (정규식 re 모듈 활용)

import re

review_nid = re.findall('\d{7}', review_nid)[0]
review_nid

'4209726'

In [ ]:
# URL을 조합합니다.
# 예시: https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4808973&code=18847&order=#tab

review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=18847&order=#tab"
review_url

'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4209726&code=18847&order=#tab'

In [ ]:
# 10개의 리뷰를 반복문으로 조회하면서, 리뷰 제목(rli.title), 사용자(rli.uid), 상세 페이지 url 값을 추출합니다.
title_list = []
uid_list = []
url_list = []

for li_tag in review_list_tags:

    review_title = li_tag.find_all('a')[0].get_text()
    title_list.append(review_title)

    review_uid = li_tag.find_all('a')[1].get_text()
    uid_list.append(review_uid)

    review_nid = re.findall('\d{7}', li_tag.find('a').get('onclick'))[0]
    review_url = f"https://movie.naver.com/movie/bi/mi/reviewread.naver?nid={review_nid}&code=81888&order=#tab"
    url_list.append(review_url)

# 추출된 아이템의 수량
print(len(title_list))
print(len(uid_list))
print(len(url_list))

10
10
10


In [ ]:
# 리스트 출력
print(url_list)

['https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4209726&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4202922&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4217186&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4219063&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4257455&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4266969&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4560148&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4204309&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4209999&code=81888&order=#tab', 'https://movie.naver.com/movie/bi/mi/reviewread.naver?nid=4201777&code=81888&order=#tab']


## 4) 리뷰 상세 페이지

In [ ]:
# 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
resp_text = requests.get(url_list[0])
soup_text = BeautifulSoup(resp_text.text, 'html.parser')

# 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

# 텍스트 부분만 추출합니다.
review_text = review_text_tag.get_text()
print(review_text)


★★★★☆"사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작."픽사는 디즈니를 닮아가고, 디즈니는 픽사를 닮아간다. 2010년대의 디즈니와 픽사의 관계를 한줄로 요약하자면 아마 이럴것이다. 이것은 단순히 픽사가 [메리다와 마법의 숲]같은 프린세스물을 만들었기에 하는 소리도 아니고, 디즈니가 [주먹왕 랄프]같이 전성기의 픽사처럼 통통튀는 상상력을 토대로한 작품을 만들었기 때문에 하는 소리도 아니다. 최근 픽사는 디즈니가 줄곧 그래왔던것처럼 '가족'이나 '감정' 같은 다소 개인적인 주제로 작품에 접근하고 있는 반면에 ([인사이드 아웃]이나 [굿 다이노]), 디즈니는 전성기 시절 픽사가 그랬던것 처럼 '정치적'인 주제로 작품에 접근하고 있다. '정치적'이라는 단어가 다소 생소하게 느껴질텐데, 디즈니가 펑크 록밴드처럼 '대통령 엿먹어!' '전쟁 반대!' 이런 구호를 외친다는것이 아니라 사회 시스템에 대해 관심을 가지기 시작했다는 의미이다. 이 뜻을 설명하기 위해서는 잠시 2000년대 중반으로 돌아가서 픽사의 전성기 시절을 되돌아 보아야 한다. 그때 당시 평론가들이 픽사 스튜디오에게 내린 주된 평은 이것이였다. '가장 정치적이면서 가장 정치적이지 않은 회사.' 여러분이 재밌게 보았던 [인크레더블]이나 [월E]같은 작품들을 떠올려보자. [인크레더블]은 '히어로 패밀리'라는 큰 주제를 가지고 있지만 은연중에 고객들을 등쳐먹고 있는 보험회사를 비판하기도 하고, [월E]에서는 대기업의 독점이 사회 시스템에 얼마나 악영향을 미치고 있는지를 보여주고 있다. 잘 생각해보면 픽사 작품에서 '숲'은 가족영화의 형태를 띄고 있지만 숲을 구성하고 있는 '나무'들은 사회를 바라보는 시니컬한 시선의 주제들로 구성되어있다. 픽사는 그런 영화를 만들던 회사였다. 그런데 픽사가 디즈니의 산하회사로 들어가게된 이후 픽사와 디즈니간의 교류가 잦아지면서 점차 둘을 구분하던 경계선이 흐릿해졌다. 이제는 픽사가 디즈니처럼 보이고, 디즈니가 픽사처럼 보이는 지경까지 이르게 된것이다. 하지만 놀랍게도 서

## 5) 10개의 리뷰 본문을 모두 수집 -> List로 정리

In [ ]:
# url 반복하여 텍스트를 추출하고 리스트에 추가

text_list = []

for url in url_list:

    # 리뷰 상세페이지의 HTML 소스코드를 가져와서 파싱(parsing)
    resp_text = requests.get(url)
    soup_text = BeautifulSoup(resp_text.text, 'html.parser')

    # 리뷰 본문의 텍스트를 추출합니다. /  <div class="user_tx_area"> )
    review_text_tag = soup_text.find(name='div', attrs={'class':'user_tx_area'})

    # 텍스트 부분만 추출합니다.
    review_text = review_text_tag.get_text()
    text_list.append(review_text)

    # 대기 시간을 추가합니다. (서버에 과도한 호출이 되지 않도록 유의)
    time.sleep(1)


# 추출된 아이템의 수량
print(len(text_list))

10


## 6) 판다스 데이터프레임으로 정리하고, CSV 파일로 저장

In [ ]:
# 딕셔너리 형식으로 항목별 리스트를 원소로 추가
dict_data = {
    'title' : title_list,
    'user' : uid_list,
    'review' : text_list
}

# 판다스 데이터프레임으로 변환
import pandas as pd

df_data = pd.DataFrame(dict_data)

# 변환 결과를 확인
df_data

,title,user,review
0,주토피아 (Zootopia),trav****,"\n★★★★☆""사회문제조차도 따뜻하게 포용하는 디즈니의 또다른 걸작.""픽사는 디즈니..."
1,주토피아 시사회 후기 - '편견'과 '다르다' 대해서 (스포주의),dlax****,\n※글 작성 관련하여 스포가 조금 있으니 피하고자 하시는 분은 뒤로가기를...--...
2,[영화/시사회] 주토피아 _ 세상은 생각보다 단순하다,dbw8****,\n​​첫인상은 솔직히 별로였다. 행여나 끝나는 순간까지 '노력으로 환경을 이겨내는...
3,주토피아를 보고 드는 의문점 (중간에 스포),mira****,"\n우선, 영화는 영상과 설정과 케릭터만 매력적이고 스토리는 그닥... 이라는 생각..."
4,불평등을 인정하고 내 삶을 능동적으로 창조하자,dldl****,\n취미로 글쓰는 중 입니다~ 많이 미흡해도 이쁘게 봐주세요!!한 사람의 인생을 멀...
5,"주토피아를 보고(결말 있음),",nasc****,\n\n당근 농장이 있는 촌에 살고 있던 토끼는세상을 좋게 만들겠다는 꿈을 갖고이상...
6,"이제훈의 영화 탐정 홍길동: 사라진 마을보다 더 재미있는, 지니퍼 굿윈의 영화 주토...",pjwo****,"\n 지니퍼 굿윈의 영화 주토피아는, 정말 잘 만든 애니메이션이다. 지니퍼 굿윈..."
7,주토피아,2jou****,"\n\n\n\n\n주토피아\n\n감독\n바이론 하워드, 리치 무어\n출연\n지니퍼 ..."
8,"<주토피아 , 2016> 틀림이 아닌 다름을 말하는 버디무비",jong****,"\n지니퍼 굿윈(주디 홉스), 제이슨 베이트먼(닉 와일드), 샤키라(가젤), 이드리..."
9,[영화시사회] 겨울왕국 & 빅 히어로 제작진의 추격전의 신세계 - <주토피아>,joun****,\n영화 시사회에 다녀왔다.뭔가 검사외전의 애니메이션 버전일 것 같은정의의 편과 사...


In [ ]:
# CSV 파일로 저장
df_data.to_csv("zootopia.csv", index=False)